# Simple Snake game in python

We would use curses package to develop and simple snake game.

The curses library supplies a **terminal-independent screen-painting and keyboard-handling facility for text-based terminals**.

## Installing Curses

* For linux and macOS, curses is already available in the Python Standard Library.

* Curses is not available in windows python. In order to use curses install `windows-curses` package pip command:

```
pip install windows-curser
```


## Initializing a curses application

#### curses.initscr()
Initialize the library. Return a window object which represents the whole screen.

#### curses.noecho()
Usually curses applications turn off automatic echoing of keys to the screen, in order to be able to read keys and only display them under certain circumstances.

#### curses.cbreak()
Applications will also commonly need to react to keys instantly, without requiring the Enter key to be pressed; this is called cbreak mode, as opposed to the usual buffered input mode.

#### stdscr.keypad(True)
Terminals usually return special keys, such as the cursor keys or navigation keys such as Page Up and Home, as a multibyte escape sequence. While you could write your application to expect such sequences and process them accordingly, curses can do it for you, returning a special value such as curses.KEY_LEFT. To get curses to do the job, you’ll have to enable keypad mode.

## Note - Copy all codes and save in python file and run it on a terminal

In [ ]:

import time
import curses

#initialize application
stdscr = curses.initscr()

# changing the echo settings
curses.noecho()
# removing the blinking cursor
curses.curs_set(0)
# adding timeout if the user doesn't respond
stdscr.timeout(3000)
# to accept other keys as input too
stdscr.keypad(True)

# writing message on screen at location (y, x)
stdscr.addstr(5, 10, "Please enter a key and wait for screen to show the response, in case you don't enter anything it will print -1")

# refreshing the screen so the text is visible
stdscr.refresh()

# waiting for user to enter a key
key = stdscr.getch()

# pause for 3 seconds so the screen is visible
time.sleep(3)

# writing message on screen at location (y, x)
stdscr.addstr(5, 10, "You entered --> " + str(key) + chr(abs(key)) + " "*100)
 
# refreshing the screen so the text is visible
stdscr.refresh()

# pause for 3 seconds so the screen is visible
time.sleep(3)

# reversing the settings
curses.echo()
curses.curs_set(1)
stdscr.keypad(False)

# close the application
curses.endwin()

## Wrapping up the application

A common problem when debugging a curses application is to get your terminal messed up when the application dies without restoring the terminal to its previous state. In Python this commonly happens when your code is buggy and raises an uncaught exception. Keys are no longer echoed to the screen when you type them, for example, which makes using the shell difficult.

We can avoid these complications and make debugging much easier by using `curses.wrapper()` function.

`curses.wrapper(func)`

Initialize curses and call another callable object, func, which should be the rest of your curses-using application.
If the application raises an exception, wrapper will restore the terminal to a sane state before re-raising the exception and generating a traceback.

Before calling func, `wrapper()`
* turns on cbreak mode,
* turns off echo,
* enables the terminal keypad, and
* initializes colors if the terminal has color support.


On exit, whether normally or by exception, it will
* restore normal mode,
* turn on echo, and
* disable the terminal keypad.

In [ ]:
import time
import curses

def main(stdscr):
    # removing the blinking cursor
    curses.curs_set(0)
    
    # writing message on screen at location (y, x)
    stdscr.addstr(5, 5, "Please enter a key and wait for screen to show the response, in case you don't enter anything it will print -1")

    # refreshing the screen so the text is visible
    stdscr.refresh()

    # waiting for user to enter a key
    key = stdscr.getch()

    # pause for 3 seconds so the screen is visible
    time.sleep(3)

    # writing message on screen at location (y, x)
    stdscr.addstr(5, 5, "You entered --> " + str(key) + chr(abs(key)) + " "*100)
    
    # refreshing the screen so the text is visible
    stdscr.refresh()

    # pause for 3 seconds so the screen is visible
    time.sleep(3)

curses.wrapper(main)

# Snake game

In [ ]:
import curses
from random import randint
import time

def main(stdscr):
    # changing settings is terminal
    curses.curs_set(0)
    stdscr.nodelay(1)
    stdscr.timeout(150)

    HEADER = " Welcome to SNAKE!! "
    KEYS_OPPOSITE_DIRECTION = {curses.KEY_UP: curses.KEY_DOWN, curses.KEY_DOWN: curses.KEY_UP, curses.KEY_RIGHT: curses.KEY_LEFT, curses.KEY_LEFT: curses.KEY_RIGHT}
    key = curses.KEY_RIGHT    
    score = 0
    

    # getting the size of the terminal
    sh, sw = stdscr.getmaxyx()
    
    # creating boundary for the game
    stdscr.border()

    # create snake and set initial direction
    snake = [[5,9], [5,8], [5,7]]
    food = [sh//2, sw//2]

    # print header, score and food
    stdscr.addstr(0, sw//2 - len(HEADER)//2, HEADER)
    stdscr.addstr(0, 2, " Score: {} ".format(score))
    stdscr.addstr(food[0], food[1], 'O')

    while 1:
        # get input from user
        event = stdscr.getch()

        # check if the user entered any of the arrow keys, if yes then assign it to key
        if event in [curses.KEY_RIGHT, curses.KEY_LEFT, curses.KEY_DOWN, curses.KEY_UP] and event != KEYS_OPPOSITE_DIRECTION[key]:
            key = event

        # get the next position based on the key
        if key == curses.KEY_RIGHT:
            snake.insert(0, [snake[0][0], snake[0][1]+1])
        elif key == curses.KEY_LEFT:
            snake.insert(0, [snake[0][0], snake[0][1]-1])
        elif key == curses.KEY_DOWN:
            snake.insert(0, [snake[0][0]+1, snake[0][1]])
        elif key == curses.KEY_UP:
            snake.insert(0, [snake[0][0]-1, snake[0][1]])

        # insert and print new head
        stdscr.addstr(snake[0][0], snake[0][1], '#')

        # check if snake is on food
        if snake[0] == food:
            # update the score
            score += 1
            stdscr.addstr(0, 2, " Score: {} ".format(score))

            # create new location for food and check it should not any of the snake's positions
            food = None
            while food is None:
                food = [randint(1, sh-2), randint(1, sw-2)]
                if food in snake:
                    food = None
            stdscr.addstr(food[0], food[1], 'O')

            # make the snake faster as it eats more
            stdscr.timeout(150 - (len(snake)//2) % 120)
        else:
            # remove the tail of the snake
            stdscr.addstr(snake[-1][0], snake[-1][1], ' ')
            snake.pop()

        # conditions for game over
        if snake[0][0] in [0, sh-1] or snake[0][1] in [0, sw-1] or snake[0] in snake[1:]:
            msg_1 = "Game Over!"
            stdscr.addstr(sh//2, sw//2-len(msg_1)//2, msg_1)
            stdscr.nodelay(0)
            time.sleep(5)
            stdscr.getch()
            break

curses.wrapper(main)